In [ ]:
import mediapipe as mp
import cv2
import pyrealsense2 as rs
import lightgbm as lgb
import datetime as dt 
import time
import csv
import os
import numpy as np
import pandas as pd
import logging
import pickle 
import sys
from math import pi
import serial
from time import sleep
import random
from sklearn.cluster import KMeans as kmc
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as lr, RidgeClassifier as rc 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier as rsc, GradientBoostingClassifier as gbc 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

Dataset Creator

In [ ]:
class_name = input("Enter the class name: ")

# Set up CSV file path
CSV_PATH = "collected_data.csv"

# Function to calculate angle between three points (A, B, C)
def calculate_angle(a, b, c):
    a = np.array(a)  # Joint A
    b = np.array(b)  # Joint B (Middle point)
    c = np.array(c)  # Joint C

    ab = a - b
    cb = c - b

    cosine_angle = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Create CSV with headers if not exists
if not os.path.exists(CSV_PATH):
    headers = ["class_name"] + \
              [f"pose_{i}_{c}" for i in range(33) for c in ['x', 'y', 'z', 'visibility']] + \
              [f"lh_{i}_{c}" for i in range(21) for c in ['x', 'y', 'z']] + \
              [f"rh_{i}_{c}" for i in range(21) for c in ['x', 'y', 'z']] + \
              [f"face_nose_{c}" for c in ['x', 'y', 'z']] + \
              [f"face_eye_l_{c}" for c in ['x', 'y', 'z']] + \
              [f"face_eye_r_{c}" for c in ['x', 'y', 'z']] + \
              [f"rh_finger_{i}_angle" for i in range(5)] + \
              [f"lh_finger_{i}_angle" for i in range(5)]
    df = pd.DataFrame(columns=headers)
    df.to_csv(CSV_PATH, index=False)

# 🟢 SET UP REALSENSE CAMERA
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 15)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 15)
# config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)
# config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
profile = pipeline.start(config)

align = rs.align(rs.stream.color)  # Align depth & color frames
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
clipping_distance = 3 / depth_scale  # Objects beyond 2m will be ignored

# Start data collection
start_time = time.time()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert frames
        color_image = np.asanyarray(color_frame.get_data())

        # Flip for natural mirror effect
        color_image = cv2.flip(color_image, 1)
        rgb_frame = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(rgb_frame)

        # Store landmark data
        landmarks = [class_name]

        # 1️⃣ Extract Pose Landmarks
        if results.pose_landmarks:
            pose_landmarks = results.pose_landmarks.landmark
            pose_data = [[lmk.x, lmk.y, lmk.z, lmk.visibility] for lmk in pose_landmarks]
        else:
            pose_data = np.zeros((33, 4))

        landmarks.extend(np.array(pose_data).flatten())

        # 2️⃣ Extract Left Hand Landmarks
        if results.left_hand_landmarks:
            left_hand_landmarks = results.left_hand_landmarks.landmark
            left_hand_data = [[lmk.x, lmk.y, lmk.z] for lmk in left_hand_landmarks]
        else:
            left_hand_data = np.zeros((21, 3))

        landmarks.extend(np.array(left_hand_data).flatten())

        # 3️⃣ Extract Right Hand Landmarks
        if results.right_hand_landmarks:
            right_hand_landmarks = results.right_hand_landmarks.landmark
            right_hand_data = [[lmk.x, lmk.y, lmk.z] for lmk in right_hand_landmarks]
        else:
            right_hand_data = np.zeros((21, 3))

        landmarks.extend(np.array(right_hand_data).flatten())

        # 4️⃣ Extract **ONLY** 3 Facial Landmarks (Nose, Left Eye, Right Eye)
        if results.face_landmarks:
            face_landmarks = results.face_landmarks.landmark
            selected_face_data = [
                [face_landmarks[1].x, face_landmarks[1].y, face_landmarks[1].z],  # Nose
                [face_landmarks[33].x, face_landmarks[33].y, face_landmarks[33].z],  # Left Eye
                [face_landmarks[263].x, face_landmarks[263].y, face_landmarks[263].z],  # Right Eye
            ]
        else:
            selected_face_data = np.zeros((3, 3))

        landmarks.extend(np.array(selected_face_data).flatten())

         # 4️⃣ Calculate Hand Angles (Right Hand)
        rh_finger_angles = []
        if results.right_hand_landmarks:
            finger_joints = [
                (0, 5, 6),   # Thumb
                (5, 6, 7),   # Index
                (6, 7, 8),   # Middle
                (10, 11, 12), # Ring
                (14, 15, 16)  # Pinky
            ]
            for (a, b, c) in finger_joints:
                rh_finger_angles.append(calculate_angle(right_hand_data[a], right_hand_data[b], right_hand_data[c]))
        else:
            rh_finger_angles = [0] * 5

        landmarks.extend(rh_finger_angles)

        # 5️⃣ Calculate Hand Angles (Left Hand)
        lh_finger_angles = []
        if results.left_hand_landmarks:
            for (a, b, c) in finger_joints:
                lh_finger_angles.append(calculate_angle(left_hand_data[a], left_hand_data[b], left_hand_data[c]))
        else:
            lh_finger_angles = [0] * 5

        landmarks.extend(lh_finger_angles)


        # Save data to CSV
        df = pd.DataFrame([landmarks])
        df.to_csv(CSV_PATH, mode='a', header=False, index=False)

        # 🟢 DRAW LANDMARKS ON FRAME
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(color_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(color_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(color_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # 🟢 ONLY DRAW NOSE + EYES, NOT FULL FACE
        if results.face_landmarks:
            keypoints = [1, 33, 263]  # Nose, Left Eye, Right Eye
            for idx in keypoints:
                landmark = results.face_landmarks.landmark[idx]
                h, w, _ = color_image.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(color_image, (x, y), 4, (0, 255, 0), -1)
        # Show countdown timer
        elapsed_time = time.time() - start_time
        remaining_time = max(0, 30 - elapsed_time)
        cv2.putText(color_image, f"Time Left: {int(remaining_time)}s", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("RealSense Landmark Detection", color_image)

        # Stop after 30 seconds or if 'q' is pressed
        if elapsed_time >= 30 or cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
pipeline.stop()
cv2.destroyAllWindows()

Reading Dataset 

-K Means Clustering

In [ ]:

df=pd.read_csv('collected_data.csv')
df.head()
df.tail()
# Extract features (exclude class labels)
X = df.drop('class_name', axis=1)
Y = df['class_name']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means clustering
optimal_clusters = 10  # You can test different values
kmeans = kmc(n_clusters=optimal_clusters, random_state=125, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataset
df['Clusters'] = clusters

# Save the new dataset
df.to_csv("collected_dataV2.csv", index=False)

# Evaluate clustering performance
silhouette_avg = silhouette_score(X_scaled, clusters)
print(f"Silhouette Score: {silhouette_avg:.4f}")  # Higher = better clustering

/home/shreyan/kyutech_project/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Pipeline & Training

In [ ]:
data=pd.read_csv("coords_NonV2.csv")
X=data.drop('class', axis=1)
Y=data['class']

# Apply K-Means Clustering (Use the same scaling as before)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = kmc(n_clusters=10, random_state=125, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to data as a new feature
X["cluster_label"] = clusters

# Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Visualize Correlation Matrix
corr_matrix = X.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.show()


In [ ]:
# Define Training Pipelines
pipelines = {
    'lr': make_pipeline(StandardScaler(), lr()), 
    'rc': make_pipeline(StandardScaler(), rc()),
    'rsc': make_pipeline(StandardScaler(), rsc()), 
    'lgb': make_pipeline(StandardScaler(), lgb.LGBMClassifier())
}


In [ ]:
# Train Models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, Y_train)
    fit_models[algo] = model 

In [ ]:
# Evaluate Models
for algo, model in fit_models.items():
    Y_hat = model.predict(X_test)
    print(algo, "Accuracy:", accuracy_score(Y_test, Y_hat))

# Use LGBMClassifier for final prediction
final_prediction = fit_models['lgb'].predict(X_test)

Saving The model


In [13]:
with open('body_language_lgbm.pkl', 'wb') as f:
    pickle.dump(fit_models['lgb'], f)

Loading and using the model for predictions

In [14]:
with open('body_language_lgbm.pkl', 'rb') as f:
    model=pickle.load(f)

Real-time prediction

In [ ]:
# Read data
rd=pd.read_csv("coords_NonV2.csv")
X = rd.drop('class', axis=1) # features
y = rd['class'] # target value
# Load trained model
with open('body_language_lgbm.pkl', 'rb') as f:
    model = pickle.load(f)

# Function to calculate angles between joints
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    cb = c - b
    cosine_angle = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# 🟢 SET UP REALSENSE CAMERA
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 15)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 15)
profile = pipeline.start(config)

align = rs.align(rs.stream.color)

# Load Mediapipe model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()

        if not color_frame:
            continue

        # Convert frames
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1)
        rgb_frame = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

        # Run Mediapipe
        results = holistic.process(rgb_frame)

        # Prepare data for prediction
        landmarks = []

        # 1️⃣ Pose Landmarks
        if results.pose_landmarks:
            pose_data = [[lmk.x, lmk.y, lmk.z, lmk.visibility] for lmk in results.pose_landmarks.landmark]
        else:
            pose_data = np.zeros((33, 4))
        landmarks.extend(np.array(pose_data).flatten())

        # 2️⃣ Left Hand Landmarks
        if results.left_hand_landmarks:
            left_hand_data = [[lmk.x, lmk.y, lmk.z] for lmk in results.left_hand_landmarks.landmark]
        else:
            left_hand_data = np.zeros((21, 3))
        landmarks.extend(np.array(left_hand_data).flatten())

        # 3️⃣ Right Hand Landmarks
        if results.right_hand_landmarks:
            right_hand_data = [[lmk.x, lmk.y, lmk.z] for lmk in results.right_hand_landmarks.landmark]
        else:
            right_hand_data = np.zeros((21, 3))
        landmarks.extend(np.array(right_hand_data).flatten())

        # 4️⃣ Facial Keypoints (Nose, Left Eye, Right Eye)
        if results.face_landmarks:
            face_landmarks = results.face_landmarks.landmark
            selected_face_data = [
                [face_landmarks[1].x, face_landmarks[1].y, face_landmarks[1].z],  # Nose
                [face_landmarks[33].x, face_landmarks[33].y, face_landmarks[33].z],  # Left Eye
                [face_landmarks[263].x, face_landmarks[263].y, face_landmarks[263].z],  # Right Eye
            ]
        else:
            selected_face_data = np.zeros((3, 3))
        landmarks.extend(np.array(selected_face_data).flatten())

        # 5️⃣ Right Hand Angles
        rh_finger_angles = []
        if results.right_hand_landmarks:
            finger_joints = [(0, 5, 6), (5, 6, 7), (6, 7, 8), (10, 11, 12), (14, 15, 16)]
            for (a, b, c) in finger_joints:
                rh_finger_angles.append(calculate_angle(right_hand_data[a], right_hand_data[b], right_hand_data[c]))
        else:
            rh_finger_angles = [0] * 5
        landmarks.extend(rh_finger_angles)

        # 6️⃣ Left Hand Angles
        lh_finger_angles = []
        if results.left_hand_landmarks:
            for (a, b, c) in finger_joints:
                lh_finger_angles.append(calculate_angle(left_hand_data[a], left_hand_data[b], left_hand_data[c]))
        else:
            lh_finger_angles = [0] * 5
        landmarks.extend(lh_finger_angles)

        # Convert to DataFrame
        X = pd.DataFrame([landmarks])

        # 🔥 Predict class
        prediction = model.predict(X)[0]
        prediction_prob = model.predict_proba(X)[0]
        max_prediction_prob = round(prediction_prob[np.argmax(prediction_prob)])

        # 🟢 Display prediction on screen
        cv2.putText(color_image, f"Prediction: {prediction}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        # 🟢 Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(color_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(color_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(color_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # 🟢 Draw key face landmarks
        if results.face_landmarks:
            keypoints = [1, 33, 263]
            for idx in keypoints:
                landmark = results.face_landmarks.landmark[idx]
                h, w, _ = color_image.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(color_image, (x, y), 4, (0, 255, 0), -1)

        cv2.imshow("Real-Time Prediction", color_image)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Cleanup
pipeline.stop()
cv2.destroyAllWindows()
